In [2]:
import numpy as np
import pandas as pd

In [3]:
fake_news = pd.read_csv('FakeNews/Fake.csv')
true_news = pd.read_csv('FakeNews/True.csv')

In [4]:
fake_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


In [5]:
true_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [6]:
fake_news.describe()

,title,text,subject,date
count,23481,23481,23481,23481
unique,17903,17455,6,1681
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017"
freq,6,626,9050,46


In [26]:
from dataset import PreProcessor
from sklearn.feature_extraction.text import TfidfVectorizer

fake_news.head()
fake_news.title[0]
preproc = PreProcessor()
x = preproc.forward(fake_news.title)

x.apply(lambda x: " ".join(x))

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(x)
x.toarray()

TypeError: expected string or bytes-like object

In [8]:
fake_news.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [9]:
true_news.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [1]:
from data import PreProcessor

preproc = PreProcessor()
fake_news['title'] = fake_news['title'].apply(lambda x: preproc.forward(x))
# fake_news['text'].apply(lambda x: preproc.forward(x))
# true_news['title'].apply(preproc.forward)
# true_news['text'].apply(preproc.forward)
fake_news['title'].head()
#ERRORS HERE

2021-11-01 08:53:12.972083: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-01 08:53:12.972113: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


NameError: name 'fake_news' is not defined

In [55]:
## Preprocessing here again CAUSE THE ABOVE CELL GOING CRAZY FOR SOME REASON

import spacy
from nltk.corpus import stopwords
import re

nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def remove_special_chars(doc):

    doc = " ".join(re.findall(r'[a-zA-Z0-9]+', doc))
    return doc.lower()

def lemmatize(doc):
    doc = nlp(doc)
    tokenized = [token.lemma_ for token in doc]
    return tokenized


def remove_stop_words(tokenized_doc):

    tokenized_doc_no_stopwords = []

    for word in tokenized_doc:
        if word not in stop_words:
            tokenized_doc_no_stopwords.append(word)

    return tokenized_doc_no_stopwords

def forward(doc):
    doc = remove_special_chars(doc)
    doc = lemmatize(doc)
    doc = remove_stop_words(doc)

    return " ".join(doc)



fake_news['title'].apply(forward)
fake_news['text'].apply(forward)
true_news['title'].apply(forward)
true_news['text'].apply(forward)
fake_news.head()

KeyboardInterrupt: 

In [45]:
fake_news = fake_news.assign(is_fake = [1]*len(fake_news))
true_news = true_news.assign(is_fake = [0]*len(true_news))

In [48]:
true_news
fake_news

,title,text,subject,date,is_fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


In [51]:
from sklearn.model_selection import train_test_split

train_fake_X, t_fake_X, train_fake_y, t_fake_y = train_test_split(fake_news.drop('is_fake', axis=1), fake_news['is_fake'], test_size=0.2, random_state=101)
test_fake_X, val_fake_X, test_fake_y, val_fake_y = train_test_split(t_fake_X, t_fake_y, test_size=0.5, random_state=101)

train_true_X, t_true_X, train_true_y, t_true_y = train_test_split(true_news.drop('is_fake', axis=1), true_news['is_fake'], test_size=0.2, random_state=101)
test_true_X, val_true_X, test_true_y, val_true_y = train_test_split(t_true_X, t_true_y, test_size=0.5, random_state=101)

In [53]:
# Joining fake and true dataframes

train_X = pd.concat([train_fake_X, train_true_X])
val_X = pd.concat([val_fake_X, val_true_X])
test_X = pd.concat([test_fake_X, test_true_X])

train_y = pd.concat([train_fake_y, train_true_y])
val_y = pd.concat([val_fake_y, val_true_y])
test_y = pd.concat([test_fake_y, test_true_y])

,title,text,subject,date
3211,Here’s The List Of Items Trump Banned From In...,Remember when the National Rifle Association w...,News,"December 31, 2016"
12353,ALMOST 100 YEARS LATER…Donald Trump Sounds A L...,,politics,"Nov 19, 2016"
291,Trump Is Too Incompetent To Handle Disaster O...,Trump has spent an awful lot of time whining a...,News,"September 29, 2017"
5856,The GOP Didn’t Waste Any Time Going Back To H...,Only days after the worst mass shooting in the...,News,"June 15, 2016"
9082,HUH? RAPPER JAY-Z Compares Trayvon Martin to M...,Rapper Jay-Z attended the Peace Walk & Peace T...,politics,15-Feb-18
...,...,...,...,...
5695,AZ Pastor: ‘Perfect Christians’ Would Bathe I...,Pastor Steven Anderson of Faithful Word Baptis...,News,"June 28, 2016"
8006,Unhinged Cruz Accuses Obama Of Being An Athei...,As one of Antonin Scalia s last rulings on the...,News,"February 18, 2016"
17745,HILARIOUS! Paul Joseph Watson DESTROYS Media F...,Paul Joseph Watson nails it again. In the vide...,left-news,"Oct 27, 2017"
17931,PRESIDENT TRUMP SLAM DUNKS Golden State NBA Pu...,"In June of 2017, it was reported that the newl...",left-news,"Sep 23, 2017"


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer